## New Ideas

1. Two RL agents instead of one.

    a. Agent one controls the movements of the robot arms.
    
        Environment: state of the robot: q and q dot
        Action: q dot dot
        Final reward: 1 if the ball is close to the busket, 0 otherwise
        ???Shaping reward: feedback from agent two if available???
    
    b. Agent two controls when to release the ball.
    
        Environment: state of the robot, hoop position
        Action: release or hold
        Final reward: 1 if the ball is close to the busket, 0 otherwise
        ???Shaping reward: it ???
    
    They have a shared goal of throwing the ball into the busket. 
    
2. While creating the training data set, positive trajectories should have higher weight.

3. The threshold should increase over time based on the performance measure of the robot so that it could continue improving.

# possible extensions

1. Add obstacles to the environment --> add constraints on the path to the goal

2. How to let the robot effectively/efficiently learn to throw toward to different positions using previous throwing experience? 

3. Encourage the robot to complete the task as fast as possible. 

4. How to quickly learn to throw the balls into a moving busket after learning fixed busket

In [1]:
# import libraries
from vpython import *
import numpy as np
import robotics_library as rbl

ImportError: No module named vpython

In [3]:
def draw_background():
    scene = canvas(background=color.black, up=vector(-1, -1, 0), height=600, center=vector(0, 0, 0))
    scene.camera.pos = vector(3.5, 3.5, 10)
    scene.camera.axis = vector(-3.5, -3.5, -10)
    
    # plot the axises
    axis_length = 3
    x = arrow(pos=vector(0,0,0), axis=vector(axis_length,0,0), shaftwidth=0.1, color=color.red)
    text(text='x', pos=vector(axis_length,0,0), axis=vector(axis_length,0,0))
    y = arrow(pos=vector(0,0,0), axis=vector(0,axis_length,0), shaftwidth=0.1, color=color.green)
    text(text='y', pos=vector(0,axis_length,0), axis=vector(0,axis_length,0))
    z = arrow(pos=vector(0,0,0), axis=vector(0,0,-axis_length), shaftwidth=0.1, color=color.blue)
    text(text='-z', pos=vector(0,0,-axis_length), axis=vector(0,0,-axis_length))
    return(scene)

def draw_robotics(joint_locs, objects2delete=[]):
    
    # delete the old positions
    for obj in objects2delete:
        obj.visible = False
        del obj 

    # color list
    color_list = [color.cyan, color.yellow, color.magenta, color.orange, vector(0.7, 0.2, 0.1),
                  vector(0.2, 0.1, 0.7), vector(0.7, 0.7, 0.8)]
    
    # draw the links
    curve_list = [[] for _ in xrange(len(joint_locs))] # container to store the objects that make up the robot
    curve_list[0] = curve(pos=[vector(0, 0, 0), vector(*joint_locs[0, :3])], radius=0.5)

    for idx in xrange(1, len(joint_locs), 1):
        p0 = vector(*joint_locs[idx-1, :3])
        p1 = vector(*joint_locs[idx, :3])
        curve_list[idx] = curve(pos=[p0, p1], color=color_list[idx])
        
    end_effector = box(pos=vector(*joint_locs[-1, :3]), length=0.7, height=0.2, width=0.3)
    curve_list.append(end_effector)

    return(curve_list)

In [4]:
env_name = "2d"
env_obj = rbl.env_dict[env_name]

In [5]:
link_lengthes = [1, 3, 1, 1, 1, 1, 1]
initial_angles = [0, 0, 0, 0, np.pi, 0]
initial_angular_velocities = np.zeros(env_obj.num_joints)
initial_rm = rbl.Robotic_Manipulator_Naive(link_lengthes, initial_angles, initial_angular_velocities)

In [6]:
q_obj = rbl.get_q_func([19, 10, 1])
# train the q_value function object

In [ ]:
q_obj = rbl.neural_fitted_q_algorithm(initial_rm, q_obj, env_obj, num_iterations=1000)

Epoch 1/3
201/201 [==============================] - 0s - loss: 0.2427     
Epoch 2/3
201/201 [==============================] - 0s - loss: 0.2393     
Epoch 3/3
201/201 [==============================] - 0s - loss: 0.2371     
Epoch 1/3
202/202 [==============================] - 0s - loss: 0.2287     
Epoch 2/3
202/202 [==============================] - 0s - loss: 0.2253     
Epoch 3/3
202/202 [==============================] - 0s - loss: 0.2218     
Epoch 1/3
210/210 [==============================] - 0s - loss: 0.2162     
Epoch 2/3
210/210 [==============================] - 0s - loss: 0.2120     
Epoch 3/3
210/210 [==============================] - 0s - loss: 0.2078     
Epoch 1/3
201/201 [==============================] - 0s - loss: 0.2024     
Epoch 2/3
201/201 [==============================] - 0s - loss: 0.1977     
Epoch 3/3
201/201 [==============================] - 0s - loss: 0.1935     
Epoch 1/3
201/201 [==============================] - 0s - loss: 0.1824     
Epoch 2/3
20

In [8]:
print(initial_rm.loc_joints())

[[ 0.  0.  1.  1.]
 [ 0.  0.  4.  1.]
 [ 0.  1.  4.  1.]
 [ 0.  2.  4.  1.]
 [ 0.  3.  4.  1.]
 [ 1.  3.  4.  1.]
 [ 2.  3.  4.  1.]]


In [29]:
def throw_the_ball(ini_robot_obj, q_obj, env_obj, discounting=0.9):
    # create a new object for plotting
    link_lengthes = ini_robot_obj.link_lengthes
    initial_angles = ini_robot_obj.joint_angles
    initial_angular_velocities = ini_robot_obj.angular_velocities
    robotic_arm = rbl.Robotic_Manipulator_Naive(link_lengthes, initial_angles, initial_angular_velocities)
    
    # create the policy object based on the q function
    policy_obj = rbl.Policy_Object(q_obj, env_obj)
    
    # start
    state = robotic_arm.state
    action = policy_obj.softmax_policy(state[:-1])
    print(state)
    print(action)
    state_list = [state]
    action_list = [action]
    
    Y = []
    # plot the background
    draw_background()
    objs2delete = draw_robotics(robotic_arm.joint_abs_locations)
    while not robotic_arm.release:
        tmp_x = np.concatenate((state[:-1], action))

        robotic_arm.update_rm(action)
        robotic_arm.loc_joints()
        objs2delete = draw_robotics(robotic_arm.joint_abs_locations, objs2delete)
        next_state = robotic_arm.state

        reward = rbl.reward_function(robotic_arm, alpha=1, env_obj=env_obj)

        next_action = policy_obj.softmax_policy(next_state[:-1])
        tmp_y = reward + discounting * q_obj.predict(np.concatenate((next_state[:-1], next_action))[np.newaxis, :])[0, 0]
        Y.append(tmp_y)

        state = next_state
        action = next_action
        print(state)
        print(action)
        state_list.append(state)
        action_list.append(action)


In [32]:
throw_the_ball(initial_rm, q_obj, env_obj)

[ 0.          0.          0.          0.          3.14159265  0.          0.
  0.          0.          0.         -0.27576202  0.67369709  0.        ]
[ 0.          0.          0.          0.         -0.00349066  0.00349066
  1.        ]


<IPython.core.display.Javascript object>

[ 0.          0.          0.          0.          2.86583063  0.67369709
  0.          0.          0.          0.         -0.27925268  0.67718775
  1.        ]
[ 0.          0.          0.          0.          0.00349066  0.          0.        ]


## move the robot by applying acceleration

In [10]:
# apply acceleration joint by joint
def test_accelerate_joint(joint_idx):
    num_accelerations = 20
    # first move in the 
    acceleration_1st = np.zeros((num_accelerations, len(rbl.action_spaces)))
    acceleration_1st[:, joint_idx] = rbl.acceleration_resolution

    acceleration_2nd = np.zeros((num_accelerations * 4, len(rbl.action_spaces)))
    acceleration_2nd[:, joint_idx] = -1 * rbl.acceleration_resolution
    
    acceleration_3rd = np.zeros((num_accelerations * 3, len(rbl.action_spaces)))
    acceleration_3rd[:, joint_idx] = rbl.acceleration_resolution
    return(np.vstack([acceleration_1st, acceleration_2nd, acceleration_3rd]))

In [11]:
rm = initial_rm
draw_background()
objs2delete = draw_robotics(rm.joint_abs_locations)
for idx in range(num_rotations):
    acceleration_matrix = test_accelerate_joint(idx)
    for action in acceleration_matrix:
        rate(20)
        rm.update_rm(action)
#         print(rm.angular_velocities)
        rm.loc_joints()
        objs2delete = draw_robotics(rm.joint_abs_locations, objs2delete)

<IPython.core.display.Javascript object>

In [14]:
print(rbl.action_spaces)

[[0.001, 0, -0.001], [0.001, 0, -0.001], [0.001, 0, -0.001], [0.001, 0, -0.001], [0.001, 0, -0.001], [0.001, 0, -0.001], [0, 1]]


In [13]:
print(rbl.action_combinations)

[[ 0.001  0.001  0.001 ...,  0.001  0.001  0.   ]
 [ 0.001  0.001  0.001 ...,  0.001  0.001  1.   ]
 [ 0.001  0.001  0.001 ...,  0.001  0.     0.   ]
 ..., 
 [-0.001 -0.001 -0.001 ..., -0.001  0.     1.   ]
 [-0.001 -0.001 -0.001 ..., -0.001 -0.001  0.   ]
 [-0.001 -0.001 -0.001 ..., -0.001 -0.001  1.   ]]


In [ ]:
print(rbl.ext_)

### joint by joint movements

In [4]:
max_rotation = np.pi / 2
num_steps = 20
rotation_angles = np.concatenate((np.linspace(0, max_rotation, num=num_steps), 
                                np.linspace(max_rotation, 0, num=num_steps),
                                np.linspace(0, -max_rotation, num=num_steps),
                                np.linspace(-max_rotation, 0, num=num_steps)))
num_angles = len(rotation_angles)
# draw initial configuration of the robot
qs = np.zeros(num_rotations)
rm.configure_robots(qs)
draw_background()
objs2delete = draw_robotics(rm.joint_abs_locations)

for idx in xrange(num_rotations):
    for idj in xrange(num_angles):
        rate(20)
        qs[idx] = rotation_angles[idj]
        rm.configure_robots(qs)
        objs2delete = draw_robotics(rm.joint_abs_locations, objs2delete)

<IPython.core.display.Javascript object>

### random movements

In [5]:
target_angles = (np.random.rand(num_rotations) - 0.5) * np.pi
target_angles[0] = np.pi / 2
num_steps = 200
delta_angles = target_angles / num_steps

# draw initial configuration of the robot
qs = np.zeros(num_rotations)
rm.configure_robots(qs)
draw_background()
objs2delete = draw_robotics(rm.joint_abs_locations)

# draw the movements
for idx in xrange(num_steps):
    rate(20)
    qs += delta_angles
    rm.configure_robots(qs)
    objs2delete = draw_robotics(rm.joint_abs_locations, objs2delete)

<IPython.core.display.Javascript object>

### input the rotation angles and display

In [9]:
rm = rbl.Robotic_Manipulator_Naive(link_lengthes, initial_angles)
draw_background()
objs = draw_robotics(rm.joint_abs_locations)
qs = [np.pi/4, 0, 0, 0, 0, 0]
rm.configure_robots(qs)
objs = draw_robotics(rm.joint_abs_locations, objs)

<IPython.core.display.Javascript object>

### location of the end effector

In [10]:
print(rm.joint_abs_locations[-1])

[-3.53553391  0.70710678  4.          1.        ]



# Questions:

1. How to calculate the velocity of the end-effector? 

    a. Use Jacobians for differential kinematics. ---> How to calculate the Jacobian matrix in the real world?
    
    b. Use Q-learning 


# implement RL for the throwing task

1. State and actions: 

    a. State: 
        
        i. position (q), velocity (v), and acceleration (a) of the joints
        
        ii. state of the end effector (hold/release)
        
        iii. terminal state: e=release
    
    b. action:
    
        i. acceleration of the joints
        
        ii. release/hold
        
    c. environment:
    
        i. location of the target busket
        
        i. initial configuration of the robot: joint angles, 0 velocity & acceleration
        
        i. position (x), velocity of the end-effector
    
2. Fixed the first 4 joints and restrain the position of the ball to be on the xz plane for simplicity. 

    a. In this case, there're only 3 actions: rotation of the last 2 joints and release/hold at the end-effector. 
    
3. Problems to solve:

    a. Technical problems:
        
        i. convert from joint positions into absolte positions, velocities, and acceleration
        
        ii. how to calculate and represent the moving direction of the end-effector
        
        iii. what algorithm to use for value function approximation? 
    
    b. RL problems:
    
        i. 